<a href="https://colab.research.google.com/github/jinkyukim-me/Summary-Seocho-Pytorch/blob/master/Simple_AutoEncoder_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated!')

from google.colab import drive
drive.mount('/content/gdrive')
print('Mounted!')

Authenticated!
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Mounted!


In [0]:
import os
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')

In [0]:
if torch.cuda.is_available():
    use_gpu = True

In [0]:
num_epochs = 100
batch_size = 128
learning_rate = 1e-3

img_transfrom = transforms.Compose([transforms.ToTensor()])

dataset = MNIST('./data', transform=img_transfrom, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [0]:
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

In [0]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
                       nn.Linear(28 * 28, 128),
                       nn.ReLU(True),
                       nn.Linear(128, 64),
                       nn. ReLU(True),
                       nn.Linear(64, 12),
                       nn.ReLU(True),
                       nn.Linear(12, 3))
        
        self.decoder = nn.Sequential(
                       nn.Linear(3, 12),
                       nn.ReLU(True),
                       nn.Linear(12, 64),
                       nn. ReLU(True),
                       nn.Linear(64, 128),
                       nn.ReLU(True),
                       nn.Linear(128, 28 * 28),
                       nn.Tanh())
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [0]:
model = autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [0]:
for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img).cuda()
        
        output = model(img)
        loss = criterion(output, img)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, loss.item()))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, '/content/gdrive/My Drive/mlp_img/image_{}.png'.format(epoch))
torch.save(model.state_dict(), '/sim_autoencoder.pth')

epoch [1/100], loss:0.0367
epoch [2/100], loss:0.0361
epoch [3/100], loss:0.0379
epoch [4/100], loss:0.0375
epoch [5/100], loss:0.0390
epoch [6/100], loss:0.0386
epoch [7/100], loss:0.0347
epoch [8/100], loss:0.0363
epoch [9/100], loss:0.0335
epoch [10/100], loss:0.0375
epoch [11/100], loss:0.0366
epoch [12/100], loss:0.0354
epoch [13/100], loss:0.0328
epoch [14/100], loss:0.0333
epoch [15/100], loss:0.0335
epoch [16/100], loss:0.0340
epoch [17/100], loss:0.0353
epoch [18/100], loss:0.0320
epoch [19/100], loss:0.0314
epoch [20/100], loss:0.0360
epoch [21/100], loss:0.0361
epoch [22/100], loss:0.0337
epoch [23/100], loss:0.0332
epoch [24/100], loss:0.0345
epoch [25/100], loss:0.0333
epoch [26/100], loss:0.0298
epoch [27/100], loss:0.0331
epoch [28/100], loss:0.0314
epoch [29/100], loss:0.0346
epoch [30/100], loss:0.0337
epoch [31/100], loss:0.0389
epoch [32/100], loss:0.0315
epoch [33/100], loss:0.0357
epoch [34/100], loss:0.0342
epoch [35/100], loss:0.0349
epoch [36/100], loss:0.0336
e